In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix, roc_curve, accuracy_score
import json
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from collections import Counter
import lightgbm as lgb
from lightgbm import early_stopping
import matplotlib.pyplot as plt
import joblib
import warnings
import os
from tqdm import tqdm
from lightgbm import record_evaluation
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV


warnings.filterwarnings('ignore')

In [2]:
def clean_feature_names(df):
    """
    Fully compatible function to remove parentheses
    """
    # Define cleaning function - uses regex to remove all types of parentheses
    def clean_name(name):
        # Use regex to remove all types of parentheses
        cleaned = name.replace('(', '').replace(')', '').replace(',', '')
        return cleaned
    
    # Apply cleaning function to all column names
    new_columns = [clean_name(col) for col in df.columns]
    df.columns = new_columns
    return df

def add_time_features(
    df: pd.DataFrame,
    id_col: str = 'Athlete ID',
    date_col: str = 'Date',
    window_sizes: list = [7, 14, 28]
) -> pd.DataFrame:
    """
    Calculate rolling statistics for each athlete by date: mean, std, max, min, injury_rate
    window_sizes: Rolling window sizes in days or weeks
    """
    df = df.sort_values([id_col, date_col]).reset_index(drop=True)
    out = df[[id_col, date_col]].copy()
    # Select numerical columns
    num_cols = df.select_dtypes(include=[np.number]).columns.drop(['injury'], errors='ignore')

    for w in window_sizes:
        # Unit is days (for weekly level, convert to week sequence first)
        rolled = (
            df
            .groupby(id_col)[num_cols]
            .rolling(window=w, min_periods=1, closed='left')  # Use only past data
            .agg(['mean', 'std', 'max', 'min'])
            .reset_index(level=0, drop=True)
        )
        # Rename columns: col_mean_7, col_std_7...
        rolled.columns = [f"{col}_{stat}_{w}d" for col, stat in rolled.columns]
        out = pd.concat([out, rolled], axis=1)

        # Injury rate: mean of 'injury' in past window
        inj_rate = (
            df
            .groupby(id_col)['injury']
            .rolling(window=w, min_periods=1, closed='left')  # Use only past data
            .mean()
            .reset_index(level=0, drop=True)
            .rename(f"injury_rate_{w}d")
        )
        out[f"injury_rate_{w}d"] = inj_rate

    return out


def data_processing_pipeline(
    week_data: pd.DataFrame,
    day_data: pd.DataFrame,
    level: str = 'day'
) -> pd.DataFrame:
    """
    Data processing pipeline: Uses rolling statistics and lag features
    Returns DataFrame with time features, lag features, and labels
    """
    # 1. Select level
    if level == 'day':
        base = day_data.copy()
    elif level == 'week':
        base = week_data.copy()
    else:
        raise ValueError("level must be 'day' or 'week'")

    # Check required columns
    for col in ['Athlete ID', 'Date', 'injury']:
        if col not in base.columns:
            raise KeyError(f"Missing column {col} in {level}_data")

    # Convert time column to datetime: Assuming Date is day number (e.g., 1, 2, ..., 2673)
    start_date = pd.to_datetime('2019-01-01')  # Customizable start date
    base['Date'] = start_date + pd.to_timedelta(base['Date'], unit='D')

    base = base.sort_values(['Athlete ID', 'Date'])
    if level == 'week':
        base = clean_feature_names(base)
    
    # 1. Add rolling window statistical features
    time_feats = add_time_features(base)


    # 2. Add injury label
    time_feats['injury'] = base['injury'].values
    time_feats['Athlete ID'] = base['Athlete ID'].values
    time_feats['Date'] = base['Date'].values
    
    # 3. Remove ID and date columns
    X = time_feats.drop(columns=['Athlete ID','Date','injury'])

    # 4. Low variance filtering
    selector = VarianceThreshold(threshold=0.001)
    X_high_var = selector.fit_transform(X)
    cols_high = selector.get_feature_names_out(X.columns)

    # 5. Missing value imputation (median)
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X_high_var)

    processed = pd.DataFrame(X_imputed, columns=cols_high)
    processed['injury'] = time_feats['injury'].values
    processed['Athlete ID'] = time_feats['Athlete ID'].values
    processed['Date'] = time_feats['Date'].values

    return processed

In [3]:
# Plot the ROC curve and save to a file
def plotROC(val_fpr, val_tpr, val_auc, test_fpr, test_tpr, test_auc, filename):
    linewidth = 2
    plt.figure(figsize=(10, 8))
    plt.plot(val_fpr, val_tpr, color='darkorange', lw=linewidth, label=f'Validation (AUC = {val_auc:.4f})')
    plt.plot(test_fpr, test_tpr, color='darkgreen', lw=linewidth, label=f'Test (AUC = {test_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=linewidth, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def determine_best_threshold(y_true, y_prob, target_recall=0.70):
    """Select threshold based on target recall rate"""
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_precision = 0.0  # Add initialization
    highest_recall = 0
    
    for threshold in thresholds:
        y_pred = (y_prob >= threshold).astype(int)
        recall = recall_score(y_true, y_pred)
        
        # Select threshold with highest precision when reaching target recall
        if recall >= target_recall:
            precision = precision_score(y_true, y_pred)
            if precision > best_precision:
                best_precision = precision
                best_threshold = threshold
        # Record highest recall as fallback
        elif recall > highest_recall:
            highest_recall = recall
            fallback_threshold = threshold
    
    # Use highest recall if target recall cannot be reached
    if best_threshold == 0.5:
        return fallback_threshold
    
    return best_threshold


def calculate_metrics(y_true, y_prob, threshold):
    """Calculate AUC, F1, and injury risk score"""
    y_pred = (y_prob >= threshold).astype(int)
    
    # Calculate AUC
    auc = roc_auc_score(y_true, y_prob)
    
    # Calculate F1 score
    f1 = f1_score(y_true, y_pred)
    auccuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    
    # Calculate injury risk score (based on confusion matrix)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    injury_risk_score = tp / (tp + fn)  # Recall/sensitivity
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    return {
        'auc': auc,
        'f1': f1,
        'injury_risk_score': injury_risk_score,
        'accuracy': auccuracy,
        'precision': precision,
        'specificity': specificity,
        'confusion_matrix': {
            'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
        }
    }

def evaluate_and_plot_model(model_name, y_val, y_val_prob, y_test, y_test_prob, best_thresh, file_prefix):
    # ROC curve
    val_fpr, val_tpr, _ = roc_curve(y_val, y_val_prob)
    test_fpr, test_tpr, _ = roc_curve(y_test, y_test_prob)
    val_auc = roc_auc_score(y_val, y_val_prob)
    test_auc = roc_auc_score(y_test, y_test_prob)

    # Plot
    plotROC(val_fpr, val_tpr, val_auc, test_fpr, test_tpr, test_auc, f'results/{file_prefix}_roc.png')
    # Calculate metrics
    metrics_test = calculate_metrics(y_test, y_test_prob, best_thresh)
    metrics_val = calculate_metrics(y_val, y_val_prob, best_thresh)

    # Output results
    print(f"\n==== {model_name} Validation Set Performance ====")
    print(f"AUC: {metrics_val['auc']:.4f}")
    print(f"F1 Score: {metrics_val['f1']:.4f}")
    print(f"Precision: {metrics_val['precision']:.4f}")
    print(f"Accuracy: {metrics_val['accuracy']:.4f}")
    print(f"Specificity: {metrics_val['specificity']:.4f}")
    print(f"Injury risk score(Recall): {metrics_val['injury_risk_score']:.4f}")
    print(f"confusion_matrix: TN={metrics_val['confusion_matrix']['tn']}, FP={metrics_val['confusion_matrix']['fp']}, "
          f"FN={metrics_val['confusion_matrix']['fn']}, TP={metrics_val['confusion_matrix']['tp']}")
    

    print(f"\n==== {model_name} Test Set Performance ====")
    print(f"AUC: {metrics_test['auc']:.4f}")
    print(f"F1 Score: {metrics_test['f1']:.4f}")
    print(f"Precision: {metrics_test['precision']:.4f}")
    print(f"Accuracy: {metrics_test['accuracy']:.4f}")
    print(f"Specificity: {metrics_test['specificity']:.4f}")
    print(f"Injury risk score(Recall): {metrics_test['injury_risk_score']:.4f}")
    print(f"confusion_matrix: TN={metrics_test['confusion_matrix']['tn']}, FP={metrics_test['confusion_matrix']['fp']}, "
          f"FN={metrics_test['confusion_matrix']['fn']}, TP={metrics_test['confusion_matrix']['tp']}")

    return metrics_test, metrics_val

In [4]:
def plot_learning_curve(evals_result, filename):
    """Plot the change of evaluation metrics during training"""
    plt.figure(figsize=(10, 6))
    
    # Get the metric name from evaluation results
    metric_name = list(evals_result['valid'].keys())[0]
    
    # Extract metric values during training
    train_metric = evals_result['train'][metric_name]
    valid_metric = evals_result['valid'][metric_name]
    
    # Plot metric changes for training and validation sets
    plt.plot(range(1, len(train_metric) + 1), train_metric, 'b-', label='Training')
    plt.plot(range(1, len(valid_metric) + 1), valid_metric, 'r-', label='Validation')
    
    # Add labels and title
    plt.title(f'Learning Curve ({metric_name})')
    plt.xlabel('Boosting Rounds')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Save the figure
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def plot_average_learning_curve(all_evals_results, filename):
    """Plot average learning curve across all folds"""
    plt.figure(figsize=(10, 6))
    
    # Determine the maximum number of iterations across all folds
    max_rounds = min(len(evals['valid']['auc']) for evals in all_evals_results)
    
    # Prepare arrays to store average values
    avg_train = np.zeros(max_rounds)
    avg_valid = np.zeros(max_rounds)
    
    # Collect metrics from all folds
    train_metrics = []
    valid_metrics = []
    
    for evals in all_evals_results:
        train_auc = evals['train']['auc'][:max_rounds]
        valid_auc = evals['valid']['auc'][:max_rounds]
        train_metrics.append(train_auc)
        valid_metrics.append(valid_auc)
    
    # Calculate average values
    avg_train = np.mean(train_metrics, axis=0)
    avg_valid = np.mean(valid_metrics, axis=0)
    
    # Plot the curves
    plt.plot(range(1, max_rounds+1), avg_train, 'b-', label='Avg Training AUC')
    plt.plot(range(1, max_rounds+1), avg_valid, 'r-', label='Avg Validation AUC')
    
    # Add labels and title
    plt.title('Average Learning Curve (AUC)')
    plt.xlabel('Boosting Rounds')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Save the figure
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def plot_multi_metric_curves(evals_result, filename_prefix):
    """Plot curves for multiple evaluation metrics"""
    # Get all metric names
    metric_names = list(evals_result['valid'].keys())
    
    for metric in metric_names:
        plt.figure(figsize=(10, 6))
        
        # Extract metric values during training
        train_metric = evals_result['train'][metric]
        valid_metric = evals_result['valid'][metric]
        
        # Plot metric changes for training and validation sets
        plt.plot(range(1, len(train_metric) + 1), train_metric, 'b-', label=f'Training {metric}')
        plt.plot(range(1, len(valid_metric) + 1), valid_metric, 'r-', label=f'Validation {metric}')
        
        # Add labels and title
        plt.title(f'Learning Curve ({metric})')
        plt.xlabel('Boosting Rounds')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(alpha=0.3)
        
        # Save the figure
        plt.savefig(f'{filename_prefix}_{metric}.png', dpi=300, bbox_inches='tight')
        plt.close()

def plot_predicted_probability_distribution(y_true, y_prob, filename):
    """
    Plot distribution of predicted probabilities grouped by true labels
    """
    plt.figure(figsize=(12, 8))
    plt.hist([y_prob[y_true == 0], y_prob[y_true == 1]], 
             bins=50, stacked=True, 
             color=['skyblue', 'salmon'], 
             label=['Actual Negative', 'Actual Positive'])
    plt.xlabel('Predicted Probability')
    plt.ylabel('Count')
    plt.title('Predicted Probability Distribution by Actual Class')
    plt.legend()
    plt.grid(alpha=0.2)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confidence_accuracy_curve(y_true, y_prob, filename):
    """
    Plot relationship between model confidence and accuracy
    """
    # Convert to NumPy arrays to avoid indexing issues
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    
    # Sort by predicted probability
    sorted_indices = np.argsort(y_prob)
    sorted_prob = y_prob[sorted_indices]
    sorted_true = y_true[sorted_indices]
    
    # Create probability bins
    bins = np.linspace(0, 1, 21)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    accuracy = []
    confidence = []
    
    # Calculate accuracy and confidence for each bin
    for i in range(len(bins) - 1):
        low, high = bins[i], bins[i+1]
        mask = (sorted_prob >= low) & (sorted_prob < high)
        if np.sum(mask) > 0:
            bin_accuracy = np.mean(sorted_true[mask] == (sorted_prob[mask] > 0.5))
            bin_confidence = np.mean(sorted_prob[mask])
            accuracy.append(bin_accuracy)
            confidence.append(bin_confidence)
    
    # Plot the relationship
    plt.figure(figsize=(12, 8))
    plt.plot(confidence, accuracy, 'o-', color='blue', label='Model')
    plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
    plt.xlabel('Model Confidence (Mean Predicted Probability)')
    plt.ylabel('Accuracy')
    plt.title('Confidence-Accuracy Curve')
    plt.legend()
    plt.grid(alpha=0.2)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def plot_class_separation(X, y_true, y_prob, filename):
    """
    Visualize class separation using dimensionality reduction
    """
    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    plt.figure(figsize=(14, 10))
    
    # Scatter plot by true labels
    plt.subplot(2, 2, 1)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_true, cmap='coolwarm', alpha=0.6)
    plt.colorbar(scatter, label='True Label')
    plt.title('True Class Separation')
    
    # Scatter plot by predicted labels
    plt.subplot(2, 2, 2)
    y_pred = (y_prob > 0.5).astype(int)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred, cmap='coolwarm', alpha=0.6)
    plt.colorbar(scatter, label='Predicted Label')
    plt.title('Predicted Class Separation')
    
    # Scatter plot by predicted probabilities
    plt.subplot(2, 2, 3)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_prob, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter, label='Predicted Probability')
    plt.title('Predicted Probability Distribution')
    
    # Scatter plot highlighting misclassified points
    plt.subplot(2, 2, 4)
    misclassified = (y_true != y_pred)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=misclassified, cmap='bwr', alpha=0.8)
    plt.colorbar(scatter, ticks=[0, 1], label='Misclassified')
    plt.title('Misclassification Points')
    
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

In [5]:
def load_and_preprocess_data(week_data_path=None, day_data_path=None, level=None):
    """
    Load and preprocess data
    level: 'day' or 'week' level data
    """
    # Load data
    # If processed data does not exist, process data
    # If processed data exists, load data
    if  level=='day':
        data = pd.read_csv(day_data_path) if day_data_path else None
        processed_data = data_processing_pipeline(
        week_data=data if level == 'week' else None,
        day_data=data if level == 'day' else None,
        level=level
    )


    else:
        data = pd.read_csv(week_data_path) if week_data_path else None
        processed_data = data_processing_pipeline(
            week_data=data if level == 'week' else None,
            day_data=data if level == 'day' else None,
            level=level)


    # Separate features and labels
    X = processed_data.drop(columns=['injury', 'Athlete ID', 'Date'])
    y = processed_data['injury']
    
    # Time-series split for train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y,  random_state=42
    )

    return X_train, X_test, y_train, y_test, processed_data

In [ ]:
BEST_PARAMS_FILE = "results/lightgbm_best_params.json"

def save_best_params(best_params, filename=BEST_PARAMS_FILE):
    """Save best parameters to JSON file"""

    with open(filename, 'w') as f:
        json.dump(best_params, f, indent=4)
    print(f"Best parameters saved to {filename}")

def load_best_params(filename=BEST_PARAMS_FILE):
    """Load best parameters from JSON file"""

    if not os.path.exists(filename):
        print(f"No best parameters file found at {filename}")
        return None
    
    with open(filename, 'r') as f:
        params = json.load(f)
    
    print("Loaded best parameters from file")
    return params

def lightgbm_train_with_params(X_train, y_train, X_val=None, y_val=None, params=None):
    """Train LightGBM model with specified parameters"""

    # Check class distribution
    class_counts = Counter(y_train)
    print(f"Original class distribution: {dict(class_counts)}")

    # Apply sampling strategy
    sampler = SMOTE(sampling_strategy=0.5, random_state=42)
    X_res, y_res = sampler.fit_resample(X_train, y_train)
    
    # Print distribution after sampling
    print(f"Training set after sampling: Class distribution = {dict(Counter(y_res))}")
    print(f"Resampled dataset size: {len(y_res)} (Positive: {sum(y_res)}, Negative: {len(y_res)-sum(y_res)})")
    
    # Set default parameters
    default_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'random_state': 42,
        'n_jobs': -1,
        'num_leaves': 31,
        'max_depth': 9,
        'learning_rate': 0.1,
        'n_estimators': 300,
        'min_child_samples': 20,
        'reg_alpha': 0.01,
        'reg_lambda': 0.01,
        'scale_pos_weight': 2,  # Handle class imbalance
        'class_weight': 'balanced',
        'force_col_wise':True,
        'verbose':-1
    }
    
    # Update defaults if parameters are provided
    if params:
        default_params.update(params)
    
    # Create and train model
    model = lgb.LGBMClassifier(**default_params)
    # create a callback function to record evaluation results
    evals_result = {}
    callbacks = [record_evaluation(evals_result)]
    
    if X_val is not None and y_val is not None:
        # Add early stopping
        callbacks.append(early_stopping(stopping_rounds=50))
        model.fit(
            X_res, y_res,
            eval_set=[(X_res, y_res), (X_val, y_val)],  # add validation set,
            eval_names=['train', 'valid'],  # set evaluation names
            eval_metric='auc',
            callbacks=callbacks
        )
    else:
        model.fit(X_res, y_res, callbacks=callbacks)
    
    return model, evals_result

def train_model(X_train, y_train, use_best_params=False):
    """Train model (using time series cross-validation)"""
    
    # Try loading existing best parameters
    best_params = load_best_params() if use_best_params else None
    
    # Check for valid data
    if len(X_train) == 0 or len(y_train) == 0:
        print("Error: Empty training data. Cannot train model.")
        return None, None
    
    # Use time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    models = []
    scores = []
    best_fold_models = []  # Store best model from each fold

    all_evals_results = []  # store all eval results
    
    print("\n=== Time Series Cross-Validation with LightGBM ===")
    for fold_idx, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
        # Ensure validation set comes after training set (time-wise)
        if min(val_idx) <= max(train_idx):
            val_idx = val_idx[val_idx > max(train_idx)]
            if len(val_idx) == 0:
                print(f"Skipping fold {fold_idx+1} - no valid validation data")
                continue
        
        print(f"\n--- Fold {fold_idx+1} ---")
        X_fold_train, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Train LightGBM model (with early stopping)
        print("Training LightGBM model...")
        # train lightgbm model and get eval results
        lgb_model, evals_result = lightgbm_train_with_params(
            X_fold_train, 
            y_fold_train, 
            X_val, 
            y_val,
            params=best_params
        )
        
        # Evaluate on validation set
        y_val_prob = lgb_model.predict_proba(X_val)[:, 1]
        lgb_auc = roc_auc_score(y_val, y_val_prob)
        print(f"LightGBM AUC: {lgb_auc:.4f}")

        # Save model and score
        models.append(lgb_model)
        scores.append(lgb_auc)
        best_fold_models.append(lgb_model)

        # save eval results
        all_evals_results.append(evals_result)
        
        # plot learning curve of current fold
        plot_learning_curve(evals_result, f'results/fold_{fold_idx+1}_learning_curve.png')
        plot_multi_metric_curves(evals_result, f'results/fold_{fold_idx+1}')
        
    
    # Select best model (based on validation AUC)
    best_model_idx = np.argmax(scores)
    best_model = models[best_model_idx]
    print(f"\nBest model selected from fold {best_model_idx+1} with AUC: {scores[best_model_idx]:.4f}")

    plot_average_learning_curve(all_evals_results, 'results/average_learning_curve.png')
    
    # Save best parameters
    if best_params is None:
        save_best_params(best_model.get_params())
    
    return best_model, best_params, all_evals_results, best_model_idx

In [ ]:
def evaluate_final_model(model, X_train, X_test, y_train, y_test, model_name, evals_result=None):
    """Comprehensive evaluation of model performance"""

    # Obtain predicted probabilities
    y_train_prob = model.predict_proba(X_train)[:, 1]
    y_test_prob = model.predict_proba(X_test)[:, 1]
    
    # Determine optimal threshold
    best_thresh = determine_best_threshold(y_train, y_train_prob, target_recall=0.75)
    print(f"Optimal threshold for recall: {best_thresh:.4f}")
    
    # Evaluate model
    metrics_test, metrics_val = evaluate_and_plot_model(
        model_name,
        y_train,
        y_train_prob,
        y_test,
        y_test_prob,
        best_thresh,
        "injury_model"
    )
    
    # Feature importance
    feature_importances = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_  
    })
    
    # Save feature importance
    feature_importances.to_csv('results/feature_importances.csv', index=False)

    # Calculate the percentage importance
    total_importance = feature_importances['importance'].sum()
    feature_importances['pct_importance'] = (feature_importances['importance'] / total_importance * 100)
    
    # Sort by original importance (in descending order)
    feature_importances = feature_importances.sort_values('importance', ascending=False)
    
    # Save feature importance
    feature_importances.to_csv('results/feature_importances.csv', index=False)

    if evals_result:
        # 1. learning curve
        plot_learning_curve(evals_result, 'results/final_model_learning_curve.png')
    # 2. predicted_probability distribution
    plot_predicted_probability_distribution(y_test, y_test_prob, 'results/predicted_prob_distribution.png')
    # 3. confidence accuracy curve
    plot_confidence_accuracy_curve(y_test, y_test_prob, 'results/confidence_accuracy_curve.png')
    # 4. class separation
    plot_class_separation(X_test, y_test, y_test_prob, 'results/class_separation.png')
    
    # Visualize top 20 features
    plt.figure(figsize=(10, 8))

    top_features = feature_importances.head(20).sort_values('pct_importance', ascending=True)
    plt.barh(top_features['feature'], top_features['pct_importance'])
    plt.xlabel('Importance (%)')
    plt.title('Top 20 Feature Importances')
    plt.tight_layout()
    plt.savefig('results/feature_importances.png', dpi=300)
    plt.close()
    return metrics_test, metrics_val, best_thresh, feature_importances

def save_model_artifacts(model, metrics, threshold, filename):
    """Save model and related metadata"""
    
    # Save model
    joblib.dump(model, f"{filename}.pkl")
    
    # Save metadata
    metadata = {
        'threshold': threshold,
        'metrics': metrics
    }
    joblib.dump(metadata, f"{filename}_metadata.pkl")
    
    print(f"Model and metadata saved to {filename}.pkl and {filename}_metadata.pkl")

In [8]:
def main():
    # Configuration parameters
    LEVEL = 'week'  # Switchbetween 'day' and 'week' level data
    DATA_PATH_DAY = "day_approach_maskedID_timeseries.csv"  
    DATA_PATH_WEEK = "week_approach_maskedID_timeseries.csv"  

    # 1. Load and preprocess data
    print("Loading and preprocessing data...")
    X_train, X_test, y_train, y_test, processed_data = load_and_preprocess_data(
        day_data_path = DATA_PATH_DAY,
        week_data_path = DATA_PATH_WEEK, 
        level=LEVEL
    )


    # 2. Train model (optionally using existing best parameters)
    print("\n=== Model Training ===")
    model, best_params, all_evals_results, best_model_idx = train_model(X_train, y_train)
    
    # 3. Evaluate model
    print("\n=== Model Evaluation ===")
    metrics_test, metrics_val, threshold, feature_importances = evaluate_final_model(
        model, 
        X_train, 
        X_test, 
        y_train, 
        y_test,
        "Injury Prediction Model",
        evals_result=all_evals_results[best_model_idx]
    )
    
    # 4. Save model and results
    print("\n=== Saving Results ===")
    save_model_artifacts(model, metrics_test, threshold, "results/injury_prediction_model")
    
    # 5. Final performance report
    print("\n=== Final Model Performance ===")
    print(f"AUC: {metrics_test['auc']:.4f}")
    print(f"F1 Score: {metrics_test['f1']:.4f}")
    print(f"Injury Risk Score (Recall): {metrics_test['injury_risk_score']:.4f}")
    print(f"Confusion Matrix:")
    print(f"  TP: {metrics_test['confusion_matrix']['tp']}")
    print(f"  FP: {metrics_test['confusion_matrix']['fp']}")
    print(f"  FN: {metrics_test['confusion_matrix']['fn']}")
    print(f"  TN: {metrics_test['confusion_matrix']['tn']}")
    
    # 6. Save best parameters (if needed)
    if best_params:
        save_best_params(best_params)

if __name__ == "__main__":
    main()

Loading and preprocessing data...

=== Model Training ===

=== Time Series Cross-Validation with LightGBM ===

--- Fold 1 ---
Training LightGBM model...
Original class distribution: {0: 5641, 1: 67}
Training set after sampling: Class distribution = {0: 5641, 1: 2820}
Resampled dataset size: 8461 (Positive: 2820, Negative: 5641)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[50]	train's auc: 0.99996	train's binary_logloss: 0.0146026	valid's auc: 0.631906	valid's binary_logloss: 0.0840001
LightGBM AUC: 0.6319

--- Fold 2 ---
Training LightGBM model...
Original class distribution: {0: 11272, 1: 142}
Training set after sampling: Class distribution = {0: 11272, 1: 5636}
Resampled dataset size: 16908 (Positive: 5636, Negative: 11272)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	train's auc: 0.994185	train's binary_logloss: 0.327568	valid's auc: 0.698864	valid's binary_logloss: 0.368254
Li